# Hard
- Test Environment: Windows 10, py-visa, NI-VISA


## TODO:

Testing on more platforms:

- x86, ubuntu-18.04.
- raspberrypi4.
- FreeBSD.

In [1]:
import time
import mhs5200

In [5]:
signal_gen.serial.close()

In [6]:
oscilloscope_port = scope_port = 'USB0::0x1AB1::0x0588::DS1EU152500705::INSTR'

In [ ]:
import mhs5200.Enums

In [ ]:
mhs5200.Enums.SQUARE

In [ ]:
channel.amplitude

In [ ]:
with mhs5200.MHS5200(signal_gen_port) as signal_generator:
    for channel in signal_generator.channels:
        channel.amplitude=1
        channel.duty_cycle=50
        channel.frequency=1
        channel.offset=0
        channel.phase=0
        channel.wave=mhs5200.Enums.SQUARE

In [ ]:
import pyvisa
rm = pyvisa.ResourceManager()
rm.list_resources()

scope = rm.open_resource(oscilloscope_port)

In [ ]:
import uuid

In [ ]:
df = pd.DataFrame()
for frequency in np.logspace(np.log10(100), np.log10(1000000), 10):
    for amplitude in [1, 5, 10, 20]:
        result_df = test_frequency_amplitude2(frequency, amplitude, signal_gen=signal_gen, scope=scope)
        df = df.append(result_df)

In [ ]:
class Scope():
    def __init__(self, *args, oscilloscope=None, oscilloscope_port=None):
        if oscilloscope_port is not None:
            self.resource = rm.open_resource(oscilloscope_port)
        else:
            self.resource = oscilloscope
            
        for fcn in ["write", "query", "close", "query_ascii_values"]:
            setattr(self, fcn, getattr(self.resource, fcn))
        
    def __enter__(self):
        return self
    
    def __exit__(self, type, value, tb):
        t1=time.time()
        try:
            while True:
                if time.time()>t1+5:
                    break
                self.close()
                time.sleep(0.2)
        except InvalidSession:
            pass
        except:
            raise

In [ ]:
scope = Scope(oscilloscope_port=oscilloscope_port)

In [ ]:
scope = Scope(oscilloscope_port=oscilloscope_port)

In [ ]:
scope.close()

In [ ]:
import pandas as pd

In [ ]:
def test_frequency_amplitude3(frequency, amplitude, signal_gen, scope):
    for chan in signal_gen.channels:
        chan.frequency=frequency
        chan.amplitude=amplitude
        chan.phase=0
    
    period = 1/float(frequency)
    timescale="{:.20f}".format(float(period/5))
    # Configure scope
    
    scope.write(f":MEASURE:TOTAL ON")
    scope.write(f":TIMebase:SCALE {timescale}")
    for scope_channel in [1, 2]:
        scope.write(f":CHANNEL{scope_channel}:probe 1")
        scope.write(f":CHANNEL{scope_channel}:scale {amplitude/5}")
        scope.write(f":CHANNEL{scope_channel}:offset 0")
    # Configure signal generator
    for chan in signal_gen.channels:
        chan.frequency=frequency
        chan.amplitude=amplitude
        chan.offset = 0
        chan.phase=0

    df = dict()
    df["uuid"] = str(uuid.uuid4())
    df["frequency"] = frequency
    df["amplitude"] = amplitude
    for source in ["CHAN1", "CHAN2"]:
        scope.write(f":MEASURE:SOURCE {source}")
        time.sleep(1)
        for param in ['VPP',
                        'VMAX',
                        'VMIN',
                        'VAMPlitude',
                        'VTOP',
                        'VBASe',
                        'VAVerage',
                        'VRMS',
                        'OVERshoot',
                        'PREShoot',
                        'FREQuency',
                        'RISetime',
                        'FALLtime',
                        'PERiod',
                        'PWIDth',
                        'NWIDth',
                        'PDUTycycle',
                        'NDUTycycle',
                        'PDELay',
                        'NDELay',
                        'TOTal',
                        'SOURce',]:
            try:
                measured = scope.query_ascii_values(f":MEASURE:{param}?")[0]
            except:
                measured = scope.query(f":MEASURE:{param}?")[0]

            df[f"{source}_{param}"] = measured
    return pd.DataFrame(df, index=[0])

In [ ]:
df = pd.DataFrame()
with mhs5200.MHS5200(signal_gen_port) as signal_gen:
    with Scope(oscilloscope_port=oscilloscope_port) as scope:
        for frequency in [1, 10, 100]:
            for amplitude in [1, 10, 20]:
                try:
                result_df = test_frequency_amplitude3(frequency, amplitude, signal_gen=signal_gen, scope=scope)
                df = df.append(result_df)

In [ ]:
df = pd.DataFrame()
signal_gen = mhs5200.MHS5200(signal_gen_port)
scope=  Scope(oscilloscope_port=oscilloscope_port)
for frequency in [1, 10, 100]:
    for amplitude in [1, 10, 20]:
        result_df = test_frequency_amplitude3(frequency, amplitude, signal_gen=signal_gen, scope=scope)
        df = df.append(result_df)

In [ ]:
import seaborn as sns

sns.set(
    rc={
        "figure.figsize": (11, 8.5),
        "figure.dpi": 300,
        "figure.facecolor": "w",
        "figure.edgecolor": "k",
    }
)
palette = (sns.color_palette("Paired"))
sns.palplot(palette)
sns.set_palette(palette)

In [ ]:
df.groupby(["frequency", "amplitude"]).agg(0")

In [ ]:
data = scope.query_binary_values(":WAVEFORM:DATA? CHAN1")
plt.plot(data)

In [ ]:
data = scope.query_binary_values(":WAVEFORM:DATA? CHAN2")
plt.plot(data)

In [ ]:
scope.query(":ACQ:SAMP? CHANnel2")

In [ ]:
scope.query(":ACQ:MEMD?")

In [ ]:
scope.write(":ACQ:MEMD LONG")

In [ ]:
for depth in ["NORMAL", "LONG"]:
    scope.write(f":ACQ:MEMD {depth}")
    time.sleep(0.5)
    assert depth == scope.query(":ACQ:MEMD?")

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
data = scope.query_binary_values(":WAVEFORM:DATA? CHAN1", "B")
plt.plot(data)

In [ ]:
data = scope.query_binary_values(":WAVEFORM:DATA? CHAN2", "B")
plt.plot(data)

In [ ]:
scope.q

In [ ]:
?scope.query_binary_values

In [ ]:
scope.query(":WAVEFORM:POINTS:MODE?")

In [ ]:
scope.write(":WAVEFORM:DATA? CHANNEL1")
header = scope.read_raw()[:10]
header

In [ ]:
scope.write(":WAVEFORM:DATA? CHANNEL1")
data = scope.read_raw()[10:]

In [ ]:
data[0]

In [ ]:
data[0:1]

In [ ]:
data[0:2]

In [ ]:
import numpy as np

In [ ]:
np.array(56).tobytes()

In [ ]:
np.array(56).tobytes("C")

In [ ]:
np.array(56).tobytes("F")

In [ ]:
np.array(56.0).tobytes("F")

In [ ]:
np.frombuffer(np.array(56).tobytes("F"))

In [ ]:
dt = np.dtype(float)
dt = dt.newbyteorder(">")
plt.plot(np.frombuffer(data))

In [ ]:
np.frombuffer(b'\x01\x02', dtype=np.uint8)

In [ ]:
np.frombuffer(b'\x01\x02\x03\x04\x05', dtype=np.uint8, count=3)

In [ ]:
dt = np.dtype(float)
dt = dt.newbyteorder("<")
plt.plot(np.frombuffer(data))